# Data Preperation and Analysis

Author: Deeepwin   
Date: 31.08.2023   
___

### Data Preparation

Specify data source:

In [ ]:
# define path to data
DATA_PATH='/mnt/c/Users/WMART/Local_Repos/neuralangelo/datasets/mouse-2'
DATA_PATH='/mnt/c/Users/WMART/Local_Repos/neuralangelo/datasets/tanks_and_temples/Ignatius'
DATA_PATH='/mnt/c/Users/WMART/Local_Repos/neuralangelo/datasets/tanks_and_temples'
DATA_PATH='/mnt/c/Users/WMART/Local_Repos/neuralangelo/datasets/dtu/dtu_scan65'
DATA_PATH='/mnt/c/Users/WMART/Local_Repos/neuralangelo/datasets/car'
DATA_PATH='/mnt/c/Users/WMART/Local_Repos/neuralangelo/datasets/dtu'

Generate transforms.json for tnt or dtu datasets:

In [ ]:
# for tnt just use 'tanks_and_temples' path and have at least two objects in that folder
!bash projects/neuralangelo/scripts/preprocess_tnt.sh $DATA_PATH

Generate transforms.json for custom dataset:

In [ ]:
!python3 projects/neuralangelo/scripts/convert_data_to_json.py --data_dir $DATA_PATH --scene_type outdoor

In [ ]:
!python3 projects/neuralangelo/scripts/generate_config.py --sequence_name leica --data_dir $DATA_PATH --scene_type outdoor

Let's inspect the COLMAP results. First, we load the COLMAP data.

In [ ]:
import numpy as np
import torch
import k3d
import json
import plotly.graph_objs as go
from collections import OrderedDict
# Import imaginaire modules.
from projects.nerf.utils import camera, visualize
from third_party.colmap.scripts.python.read_write_model import read_model
# Read the COLMAP data.
cameras, images, points_3D = read_model(path=f"{DATA_PATH}/sparse", ext=".bin")
# Convert camera poses.
images = OrderedDict(sorted(images.items()))
qvecs = torch.from_numpy(np.stack([image.qvec for image in images.values()]))
tvecs = torch.from_numpy(np.stack([image.tvec for image in images.values()]))
Rs = camera.quaternion.q_to_R(qvecs)
poses = torch.cat([Rs, tvecs[..., None]], dim=-1)  # [N,3,4]
print(f"# images: {len(poses)}")
# Get the sparse 3D points and the colors.
xyzs = torch.from_numpy(np.stack([point.xyz for point in points_3D.values()]))
rgbs = np.stack([point.rgb for point in points_3D.values()])
rgbs_int32 = (rgbs[:, 0] * 2**16 + rgbs[:, 1] * 2**8 + rgbs[:, 2]).astype(np.uint32)
print(f"# points: {len(xyzs)}")

This is where you should visualize and adjust the bounding sphere for Neuralangelo.
- Use the forms to tune `readjust_center` and `readjust_scale` to adjust the bounding sphere.
  - The bounding sphere should ideally *just* encapsulate the target object/scene.
  - In the Lego toy example case, setting `readjust_scale=0.5` would be a good choice.
- Also check whether the camera trajectory matches the expectation from the video observation.

In [ ]:
json_fname = f"{DATA_PATH}/transforms.json"
with open(json_fname) as file:
    meta = json.load(file)
center = meta["sphere_center"]
radius = meta["sphere_radius"]
# ------------------------------------------------------------------------------------
# These variables can be adjusted to make the bounding sphere fit the region of interest.
# The adjusted values can then be set in the config as data.readjust.center and data.readjust.scale
readjust_x = 0.0  # @param {type:"number"}
readjust_y = 0.0  # @param {type:"number"}
readjust_z = 0.  # @param {type:"number"}
readjust_scale = 1.0  # @param {type:"number"}
readjust_center = np.array([readjust_x, readjust_y, readjust_z])
# ------------------------------------------------------------------------------------
center += readjust_center
radius *= readjust_scale
# Make some points to hallucinate a bounding sphere.
sphere_points = np.random.randn(100000, 3)
sphere_points = sphere_points / np.linalg.norm(sphere_points, axis=-1, keepdims=True)
sphere_points = sphere_points * radius + center

Visualize the bounding sphere in the 3D interactive visualizer.
- If the bounding sphere doesn't look right, readjust in the above form and rerun the code block.
- You can modify `vis_depth` to adjust the size of the cameras.

In [ ]:
# @title { vertical-output: true }
vis_depth = 0.2
# Visualize with Plotly.
x, y, z = *xyzs.T,
colors = rgbs / 255.0
sphere_x, sphere_y, sphere_z = *sphere_points.T,
sphere_colors = ["#4488ff"] * len(sphere_points)
traces_poses = visualize.plotly_visualize_pose(poses, vis_depth=vis_depth, xyz_length=0.02, center_size=0.01, xyz_width=0.005, mesh_opacity=0.05)
trace_points = go.Scatter3d(x=x, y=y, z=z, mode="markers", marker=dict(size=1, color=colors, opacity=1), hoverinfo="skip")
trace_sphere = go.Scatter3d(x=sphere_x, y=sphere_y, z=sphere_z, mode="markers", marker=dict(size=0.5, color=sphere_colors, opacity=0.7), hoverinfo="skip")
traces_all = traces_poses + [trace_points, trace_sphere]
layout = go.Layout(scene=dict(xaxis=dict(showspikes=False, backgroundcolor="rgba(0,0,0,0)", gridcolor="rgba(0,0,0,0.1)"),
                              yaxis=dict(showspikes=False, backgroundcolor="rgba(0,0,0,0)", gridcolor="rgba(0,0,0,0.1)"),
                              zaxis=dict(showspikes=False, backgroundcolor="rgba(0,0,0,0)", gridcolor="rgba(0,0,0,0.1)"),
                              xaxis_title="X", yaxis_title="Y", zaxis_title="Z", dragmode="orbit",
                              aspectratio=dict(x=1, y=1, z=1), aspectmode="data"), height=800)
fig = go.Figure(data=traces_all, layout=layout)
fig.show()

### Analysis of Result

In [ ]:
OUTPUT_DIR='/mnt/c/Users/WMART/Local_Repos/nerf/exports/neuralangelo-mouse-2/mouse-2_mesh.ply'
OUTPUT_DIR='/mnt/c/Users/WMART/Local_Repos/nerf/exports/neuralangelo-ignatius/ignatius_mesh.ply'
OUTPUT_DIR='/mnt/c/Users/WMART/Local_Repos/nerf/exports/nerfacto-mouse-2-nerfstudio/poisson_mesh.ply'
OUTPUT_DIR='/mnt/c/Users/WMART/Local_Repos/nerf/exports/nerfacto-big-mouse-2-nerfstudio/poisson_mesh.ply'
OUTPUT_DIR='/mnt/c/Users/WMART/Local_Repos/nerf/exports/neus-facto-dtu65/mesh.ply'
OUTPUT_DIR='/mnt/c/Users/WMART/Local_Repos/nerf/exports/neuralangelo-dtu65-sdfstudio/mesh.ply'
OUTPUT_DIR='/mnt/c/Users/WMART/Local_Repos/nerf/exports/neuralangelo-dtu65/dtu65_mesh.ply'

In [ ]:
!open3d draw $OUTPUT_DIR